In [ ]:
import json
import requests
import time
import os
import logging
from datetime import datetime

In [ ]:
OPENSEA_API_BASE = "https://api.opensea.io/api/v2"
OPENSEA_API_KEY = "" 
OPENSEA_API_BASE_V1 = "https://api.opensea.io/api/v1" 
collection_slug="doodles-official" # [pudgypenguins, chimpersnft, doodles-official]
chain="ethereum"

In [ ]:

get_collection_event_url = "https://api.opensea.io/api/v2/events/collection/{collection_slug}/?event_type=sale&limit=50&next={next_string}"

In [ ]:
def make_api_request(url, headers=None, params=None):
    """Make API request with basic rate limiting"""
    max_retries = 3
    retry_count = 0
    
    if headers is None:
        headers = {
            "Accept": "application/json",
            "X-API-KEY": OPENSEA_API_KEY
        }
    while True:
        try:
            response = requests.get(url, headers=headers, params=params)
            
            # Handle rate limiting
            if response.status_code == 429:
                print("Rate limited, waiting 1 second...")
                time.sleep(1)
                return make_api_request(url, headers, params)
            
            # Raise exception for other errors
            response.raise_for_status()
            retry_count = 0
            return response.json()
        except Exception as e:
            print(f"Error making request to {url}: {e}")
            retry_count += 1
            if retry_count >= max_retries:
                print(f"Max retries reached")
                return None
            time.sleep(2 ** retry_count)  # Exponential backoff


In [ ]:
def get_nfts_event(collection_slug,next_string=""):
    url = get_collection_event_url.format(collection_slug=collection_slug,next_string=next_string)
    return make_api_request(url)

In [ ]:

def requset_log(log_dir):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(f"{log_dir}nft_events_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"),
            logging.StreamHandler()  # Also prints to console
        ]
    )

In [ ]:
nfts_event_output_dir = f"./data/{collection_slug}/collection_sale_events/"
os.makedirs(nfts_event_output_dir, exist_ok=True)

# Initialize variables
all_events = []
all_next_strings = []
next_string = ""
page_counter = 0
save_interval = 1  # Save every 10 pages
file_counter = 1  # For sequential file naming
requset_log(nfts_event_output_dir)

while True:
    try:
        response = get_nfts_event(collection_slug=collection_slug, next_string=next_string)
        
        # Add new events to our collection
        if 'asset_events' in response:
            all_events.extend(response['asset_events'])
            all_next_strings.append(response['next'])
            page_counter += 1
            
            # Save every 10 pages or when done
            if page_counter % save_interval == 0 or ('next' not in response or not response['next']):
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                output_filename = f"events_part{file_counter}.json"
                output_path = os.path.join(nfts_event_output_dir, output_filename)
                
                with open(output_path, 'w') as outfile:
                    json.dump({
                        "metadata": {
                            "collection": collection_slug,
                            "pages_fetched": page_counter,
                            "this_next_string": next_string,
                            "next_string": response['next'],
                            "save_time": timestamp
                        },
                        "events": all_events,
                        "next_strings": all_next_strings
                    }, outfile, indent=2)
                
                logging.info(f"Saved {len(all_events)} events to {output_filename} (pages {page_counter - save_interval + 1}-{page_counter})")
                file_counter += 1
                all_events = []  # Reset for next batch
                all_next_strings = []  # Reset for next batch
        
        # Check for more pages
        if 'next' in response and response['next']:
            next_string = response['next']
            time.sleep(1)  # Rate limiting
        else:
            break
            
    except Exception as e:
        print(f"Error on page {page_counter}: {str(e)}")
        time.sleep(5)  # Wait before retrying
        continue

logging.info(f"Completed! Fetched {page_counter} pages total.")